In [48]:
# 나의 구글 드라이브를 구글 본사 서버와 연결하는 작업
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 현재 실행 경로 확인하기
import os
print(os.getcwd())

/content/drive/My Drive/nlp


In [ ]:
# 현재 작업 폴더 nlp로 경로 이동하기
%cd '/content/drive/My Drive/nlp'

/content/drive/My Drive/nlp


In [ ]:
print(os.getcwd())

/content/drive/My Drive/nlp


In [ ]:
# 필요한 라이브러리 임폴트
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
"""
KoNLpy(코엔엘파이) 라이브러리 설치
KoNLpy(코엔엘파이) 라이브러리 설치 : 한국어 형태소 분석기로서, 한글 자연어 처리를 위해
만들어진 라이브러리
"""

# KoNLpy 라이브러리 설치
!pip install konlpy

In [ ]:
# 필요한 라이브러리 임폴트
import konlpy
from konlpy.tag import Kkma, Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# 한글 형태소 분석기 생성

kkma = Kkma()
okt = Okt()

In [49]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
df = pd.read_csv('3rd_last_project.csv')
#pd.options.display.max_columns = None
print("df.title = \n", df.title)
print("df.content = \n", df.content)

df.title = 
 0      광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t...
1                            2021년 청년 13(일+삶통장 6기 참여자 모집
2      2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\...
3                  고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집
4                                  2021 행복플러스 건강지원 사업 안내
                             ...                        
117                                           철학으로 읽는 옛집
118             코로나 극복 예술프로젝트 ‘슬기로운 문화생활, GAC유튜브 안방예술극장’
119                         코로나19 예방접종 이상반응 피해보상 구비서류 안내
120    필리핀 전래동화 ‘바닷물은 왜 짤까?’를 바탕으로 인간과 자연의 평화로운 공존을 이...
121    이야기꽃도서관 독서문화프로그램 및 그림책 활동가 동아리 프로그램 등 기타 행사 운영...
Name: title, Length: 122, dtype: object
df.content = 
 0      {기간:2021-04-27 ~2021-09-30}\n{대상:1991.01.01~20...
1      {기간:2021-04-27 ~2021-10-20}\n{대상:1991.01.01~20...
2      {기간:2021-04-27 ~2021-10-20}\n{대상:1991.01.01~20...
3      {기간:2021-06-25 ~2021-12-13}\n{대상:1991.01.01~20...
4      {기간:2021-06-11 ~2021-12-13}\n{대상:1991.01.01~20...
                    

In [50]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    122 non-null    object
 1   content  122 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB
None


In [ ]:
# print(list(df.title))
# df_tmp1 = pd.DataFrame(list(df.title), columns=['title'])
# print(df_tmp1)

In [51]:
df_tmp = pd.DataFrame([df.title, df.content],columns=['title','content'])
print(df_tmp.head())

         title  content
title      NaN      NaN
content    NaN      NaN


In [52]:
#데이터 (타이틀) 전처리 
import re
sentences = [data for data in df.title]

for data in sentences:

  #print("data = ", data)
  re.sub('[^0-9a-zA-Zㄱ-힗]+', '', data)
  # cleaned_text.append(data)


text = sentences

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)

print("타이틀:",text)
print("타이틀갯수:",len(text))

타이틀: ['광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t\t\t\t', '2021년 청년 13(일+삶통장 6기 참여자 모집', '2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\t\t\t', '고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집', '2021 행복플러스 건강지원 사업 안내', '마음건강상담소 오픈-광주청년센터\t', '청년의 일터를 지키는 N개의 청년일기 사연모집', '광주청년드림은행 6월 가계부 워크샵 안내\t', "2021년 '청년교통수당 드림' 참여자 모집공고\t\t\t\t\t\t\t\t", '광주청년 일경험드립 사업 제10기 드림청년(참여청년 공개모집 연장공고', '의료기관아동학대, 긴급복지 신고의무자 교육 및 범죄 조회 공지', '광주글로벌모터스 AI역량검사 지원부스 운영', '광주글로벌모터스 채용설명 취업특강<7.23(금, 7.26(월> 운영 안', '[2021 국민참여 청렴콘텐츠 공모전] 2차 소문내기 이벤트 시작', '[2021 국민참여 청렴콘텐츠 공모전] 소문내기 이벤트', '사회적거리두기 2단계 격상에 따른 광산구립도서관 운영 안내', '2020년도 김대중컨벤션센터 전시장 정시모집 종료 및 수시대관 실시 안내', '[공지]2021년 12월 다목적홀 대관 공고', '[공지]2021년 광주광역시 우수전시회 지원사업 모집공고', '[공지]2021년 광주광역시 우수전시회 지원사업 재 모집 공고', '[공지]2021년도 김대중컨벤션센터 전시장 정시대관 종료 및 수시대관 실시 안내', '[공지]2022년 상반기 전시장 정시모집 공고', '[공지]2022년 상반기 전시장 정시모집 재 공고', '[공지]김대중컨벤션센터 대표홈페이지 (장기 미접속자 개인정보 파기 안내', '[공지]김대중컨벤션센터 상설전시장 및 회의실 로비 임대요율 변경 안내', '[공지]김대중컨벤션센터 임대요율 인상 안내', '[공지]김대중컨벤션센터 전시장 임대료 월별 차등율 변경 안내'

'이야기꽃도서관 독서문화프로그램 및 그림책 활동가 동아리 프로그램 등 기타 행사 운영 안내7월'

In [ ]:
#기간과 대상 추출하기 위한 사전만들기
#그외 나머지는 내용 싹 다 담아라
test_date = ['기간', '접수 기간','기간 :']

test_people = ['대상', '대상자', '대상 :', '대상자 :']

In [53]:
# #기간 대상 추출하기_test
base_data = [data for data in df.content]
#print("base_data = \n", base_data[0])
print('\n')

# #stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
sentences = []
for data in base_data:    
  sentence_str = str(data)
  sentence = sentence_str.split('\n')
  sentences.append(sentence)
print("sentences[0] = \n", sentences[0])

# # 기간
period_front = []
period_end = []
# # 대상
run = []


for sentence in sentences:
  for data in sentence:
    print("data = ", data)
    if '{기간:}' in data[:7] or '{기간}' in data[:7] or '기간:' in data[:7]:
      period_front.append(data[4:14])
      period_end.append(data[16:-1])



for sentence in sentences:
  for data in sentence:
    print("data = ", data)
    if '{대상:}' in data[:7] or '{대상}' in data[:7] or '대상:' in data[:7]:
      run.append(data[4:-1])

# print(period[0])
# print(len(period[0]))

# 결과 확인
print("기간-front", period_front)
print("기간-end", period_end)
print("대상", run)

# cleaned_period  = []
# for data in period:
#   if ':' in data:
#     cleaned_period.append(data)
#   else:
#     continue

# print(cleaned_period)


zeros = [0] * (122 - len(run))

result_period = run  + zeros


print(result_period)
print(f"기간 결과의 길이 = {len(result_period)}")



sentences[0] = 
 ['{기간:2021-04-27 ~2021-09-30}', '{대상:1991.01.01~2002.01.01}', '2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 : 청년금융복지지원센터,등록일 : 2021-04-27 08:40:13,조회수 : 1756,첨부파일 1,2021년_청년13통장_6기_선정자_공고문.hwp [13 KB],첨부파일 2,2021년_청년13통장_6기_선정자_및_예비자_명단.xlsx [37 KB],, 일하는 청년의 더 나은 삶을 위해 임금소득이 낮은 근로청년의 소액 단기 저축을 지원하는,‘2021년 청년 13(일+삶통장 지원 사업’ 선정자 명단 및 예비후보자 명단을 공고합니다,광 주 광 역 시 장 ,목록,다음글,610. [안내]마음건강상담소 오픈-광주청년센터 청년금융복지지원센터,이전글,581. [공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,청년금융복지지원센터']
data =  {기간:2021-04-27 ~2021-09-30}
data =  {대상:1991.01.01~2002.01.01}
data =  2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 : 청년금융복지지원센터,등록일 : 2021-04-27 08:40:13,조회수 : 1756,첨부파일 1,2021년_청년13통장_6기_선정자_공고문.hwp [13 KB],첨부파일 2,2021년_청년13통장_6기_선정자_및_예비자_명단.xlsx [37 KB],, 일하는 청년의 더 나은 삶을 위해 임금소득이 낮은 근로청년의 소액 단기 저축을 지원하는,‘2021년 청년 13(일+삶통장 지원 사업’ 선정자 명단 및 예비후보자 명단을 공고합니다,광 주 광 역 시 장 ,목록,다음글,610. [안내]마음건강상담소 오픈-광주청년센터 청년금융복지지원센터,이전글,581. [공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,청년금융복지지원센터
data =  {기간:2021-

In [ ]:
(len(run))

In [57]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
# df = pd.read_csv('.csv')
df2 = pd.read_csv('3rd_last_project_2.csv')

pd.options.display.max_columns = None

dict_ = {'title':list(df2.title), 'content':list(df2.content),'end_date':period_end,'target':run}

# 기간(start, end), 대상, 타이틀 4가지 데이터를 하나의 데이터프레임으로 만들기
import pandas as pd
df_cleaned = pd.DataFrame(dict_)
print("결과 확인", df_cleaned.info())
print(df_cleaned.head())

# 결과를 pickle로 저장하기
df_cleaned.to_pickle('df_cleaned.pkl')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     122 non-null    object
 1   content   122 non-null    object
 2   end_date  122 non-null    object
 3   target    122 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB
결과 확인 None
                                               title  \
0  광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t...   
1                        2021년 청년 13(일+삶통장 6기 참여자 모집   
2  2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\...   
3              고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집   
4                              2021 행복플러스 건강지원 사업 안내   

                                             content    end_date  \
0  2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 ...  2021-09-30   
1  [공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,작성자 ...  2021-10-20   
2  [안내]마음건강상담소 오픈-광주청년센터,작성자 : 청년금융복지지원센터,등록일 : 2...  2021-10-20   
3  643. 

In [58]:
# pickle 파일 형태로 저장되어 있는 df_cleaned.pkl 파일을 불러와서 읽기
pd.options.display.max_columns = None
df_cleaned = pd.read_pickle('df_cleaned.pkl')
#print("결과확인", df_cleaned)

In [59]:
df_cleaned

,title,content,end_date,target
0,광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t...,"2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 ...",2021-09-30,1991.01.01~2002.01.01
1,2021년 청년 13(일+삶통장 6기 참여자 모집,"[공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,작성자 ...",2021-10-20,1991.01.01~2002.12.31
2,2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\...,"[안내]마음건강상담소 오픈-광주청년센터,작성자 : 청년금융복지지원센터,등록일 : 2...",2021-10-20,1991.01.01~2002.12.31
3,"고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집","643. [청년지원사업안내]2021년 '청년교통수당 드림' 참여자 모집공고,작성자 ...",2021-12-13,1991.01.01~2002.12.31
4,2021 행복플러스 건강지원 사업 안내,"638. [참고]광주청년드림은행 6월 가계부 워크샵 안내,작성자 : 청년금융복지지원...",2021-12-13,1991.01.01~2002.12.31
...,...,...,...,...
117,철학으로 읽는 옛집,"토요인문학> 철학으로 읽는 옛집 기간(8/28, 9/25 ,•우리 옛집의 철학 ,•...",2021-09-25,1900.01.01~2021.01.01
118,"코로나 극복 예술프로젝트 ‘슬기로운 문화생활, GAC유튜브 안방예술극장’","광주문화예술회관에서 제안하는 슬기로운 문화생활,코로나 극복 예술프로젝트‘힘내라 대한...",2021-12-31,1900.01.01~2021.01.01
119,코로나19 예방접종 이상반응 피해보상 구비서류 안내,"코로나19 예방접종 이상반응 피해보상 구비서류 안내,,> 신청대상 : 코로나19 예...",2021-12-31,1990.01.01~2021.01.01
120,필리핀 전래동화 ‘바닷물은 왜 짤까?’를 바탕으로 인간과 자연의 평화로운 공존을 이...,"기간,2021.9.4.(토 - 9.5.(일,시간,11:00, 15:00,장소,어린이...",2021-9-5,1900.01.01~2021.01.01
